In [1]:
import re
import import_ipynb
import numpy as np
import pandas as pd
import difflib
import transformers
import spacy
from transformers import BertTokenizerFast

In [2]:
commonly_used_words = ["of","america","american","taiwan","banco","mexico","china","health","healthcare","pharma","pharmaceutical","us","partners","united states","managment","mosaic",
                       "financial","bank","companies","services","products","brands","air","airlines","international","asset","equity","fund","group","resourses","technologies","hotels","control","controls","black","green","natural","steel","motor",
                       "general","resourses","electric","payments","home","world","union","credit","business","public","shipping","capital","express","royal","mobile","microelectronics"
                       "first","exchange","block","united","energy","national","realty","york","titan","community","skin","foods","industrial","iron","paper","crown","petroleum","jewelers","communications"]

In [3]:
df_labels = pd.read_csv("news_handled/ls_new_nasdaq_labeled_companies_holdings.csv",index_col=False,header=0)
df_all_extracted_companies = pd.read_csv("news_handled/ls_new_all_companies_nasdaq_normalized.csv",index_col=False,header=0)
df_news = pd.read_csv("news_handled/nasdaq_news.csv",index_col=False,header=0)

In [24]:
nlp = spacy.load("en_core_web_sm")
COMMENT = ""
print(df_labels.head())

                                                Name  \
0                                           US Steel   
1                 Taiwan Semiconductor Manufacturing   
2                            United Microelectronics   
3                                Goldman Sachs Group   
4  American Tower\tBlackstone\tCrown Castle Inter...   

                                           Long name  
0                           United States Steel Corp  
1         Taiwan Semiconductor Manufacturing Company  
2                       United Microelectronics Corp  
3                            Goldman Sachs Group Inc  
4  American Tower Corp\tBlackstone Inc\tCrown Cas...  


In [5]:
def print_geo(real_str):
    global COMMENT
    doc = nlp(real_str)
    for ent in doc.ents: 
        if(ent.label_ == 'GPE' or ent.label_ == 'GEO'):
            print(ent.text)
            print(ent.label_)

In [6]:
print_geo("Since 1958, Winnebago has been innovating recreation–from SmartSpace design to advanced steel chassis-based safety features and everything in between–and ")

Winnebago
GPE


In [8]:
tokenizer = transformers.BertTokenizerFast.from_pretrained("dslim/bert-base-NER")

In [9]:
print(print_geo("Yum Brands claimed today than Tim and Ben was yesterday in small town called Aruba"))

NameError: name 'print_geo' is not defined

In [12]:
print(commonly_used_words[0])
print("brands" in commonly_used_words)

of
True


In [13]:
def is_valid_match(real_str, extracted_str):
    global COMMENT
    
    all_real_words_is_common = True
    all_extracted_words_is_common = True
    real_words = real_str.lower().replace("-"," ").split(" ")#for walmart and other
    extracted_words = extracted_str.lower().replace("-"," ").split(" ")
    
    
    for word in real_words:
        if(word not in commonly_used_words and len(word) > 3):
            all_real_words_is_common = False
            
    if(all_real_words_is_common):
        COMMENT = "all words in real is commonly used, only full match avaliable"
        return False
    
    
    for word in extracted_words:
        if(word not in commonly_used_words and len(word) > 3):
            all_extracted_words_is_common = False
    if(all_extracted_words_is_common):
        COMMENT = "all words in extracted company is commonly used, only full match avaliable"
        return False
    
    if(len(real_words) > 1 and len(extracted_words) > 1 and real_str.replace(" ","").find(extracted_str)!=-1 ):
        COMMENT = "Vaild words match"
        return True
    
    for word in extracted_words:
        if(word not in real_words):
            return False
    COMMENT = "Vaild words match"
    return True

In [14]:
def is_also_proper(real_str, extracted_str,nlp):
    global COMMENT
    doc = nlp(real_str)
    
    for tok in doc: 
        if(tok.pos_ == 'PROPN' and extracted_str.replace("-","").title().find(tok.text)==-1 and tok.text not in tokenizer.get_vocab() and len(tok.text)>3 and tok.text.lower not in commonly_used_words):
            return False
        if(not tok.text.isupper() and tok.text.lower() not in commonly_used_words and tok.pos_ == 'PROPN' and extracted_str.replace("-","").title().find(tok.text)!=-1 and len(tok.text)>3):
            COMMENT = "Both companies have same proper name"
            return True
    return False

In [15]:
def is_also_person(real_str, extracted_str,nlp):
    global COMMENT
    doc = nlp(real_str)
    for ent in doc.ents: 
        if(ent.label_ == 'PERSON' and extracted_str.find(ent.text)!=-1 and len(ent.label > 2)):
            COMMENT = "Both companies have same person name"
            return True
    return False

In [16]:
def is_both_unique(real_words, extracted_words):#check if organisation name contains unique word
    global COMMENT

    if(len(real_words)==1 and len(extracted_str.replace("-","").split(" "))==1):#one subword - not valid
        return False
    for word in real_words:
        if(word in extracted_words and word not in tokenizer.get_vocab() and word.lower() not in commonly_used_words):
            COMMENT = "Both companies have unique word"
            return True
    #words = real_str.split(" ")
    #for word in words:
    #    if(word in extracted_words and word not in tokenizer.get_vocab() and word.lower() not in commonly_used_words):
    #        COMMENT = "Both companies have unique word"
    #        return True
    return False

In [17]:
def is_extracted_contain_real(words, extracted_words):#check if organisation name contains person name
    #if(extracted_str.find(real_str) !=-1):
    #    return True
    for word in words:
        if(word not in extracted_words):
            return False
    return True

In [18]:
def long_name_match(words, extracted_words):#check if organisation name contains person name
    count = 0
    for word in words:
        if(word in extracted_words and word not in commonly_used_words):
            count += 1
    if(count >= 2):
        return True
        comment = "More then 1 common word with long name"
    return False

In [19]:
def is_abbreviation(real_str, extracted_str):
    global COMMENT
    company = real_str.lower()
    extracted = extracted_str.replace("-"," ").lower().split(" ")
    extracted = [elem for elem in extracted if len(elem)!=0]
    if not(extracted and company):
        return False
    if(len(company)!=len(extracted)): 
        if(company[-1] == 'c' or company[-1] == 'g'):
            extracted.append(company[-1])
    if(len(company)!=len(extracted)):
        return False
    
    for idx in range(len(extracted)):
        if(company[idx]!=extracted[idx][0]):
            return False
    COMMENT = "Company is abbreviation"
    return True

In [20]:
def is_complicated_abbreviation(abbrev_str, full_str): #FedEx is Federal Express for example
    short_words = re.findall('[A-Z][^A-Z]*', abbrev_str)
    full_words = full_str.split()
    if(len(short_words)!=len(full_words)  or len(short_words) < 2): 
        return False
    for idx in range(len(short_words)):
        if(full_words[idx].find(short_words[idx]) == -1):
            return False
    COMMENT = "Company is complicated abbreviation"
    return True

In [21]:
def common_mixed_word(real_words, extracted_words):#check if organisation name contains unique word
    global COMMENT
    if(len(real_words)==1 and len(extracted_str.replace("-","").split(" "))==1):#one subword - not valid
        return False
    for word in real_words:
        if(word in extracted_words  and not word.isupper() and not word.islower() and not word.istitle()):
            COMMENT = "Both companies common mixed word"
            return True

In [22]:
def only_commonly_used_is_common(real_words, extracted_words):
    isOkay = False
    for word in real_words:
        if(word in extracted_words and word not in commonly_used_words):
            return False
        elif(word in extracted_words and word in commonly_used_words):
            isOkay = True
    return isOkay

In [ ]:
print(is_valid_match("Bank of america","BofA merril linch"))
print(is_complicated_abbreviation("Chipotle","Chipotle Mexican Grill"))
print(is_complicated_abbreviation("FedEx","Federal Express"))
print(is_valid_match("American International","American International Group"))
print(is_extracted_contain_real("Alcoa","Gary Hill"))

In [21]:
print("Nemours" in tokenizer.get_vocab())
print(is_valid_match("Yum! Brands","Jerry Dean"))
print(is_also_proper("BNY Mellon","Bank of New York Mellon",nlp))
print(is_both_unique("Yum Brands","Conagra Brands"))
print(is_both_unique("Sl Green Realty","Slg"))
print(is_both_unique("Dupont De Nemours","Ei Du Pont De Nemours"))
print(is_valid_match("Jp Morgan Chase","Jpmorgan"))
print(is_extracted_contain_real("Alcoa","Gary Hill"))
print(is_valid_match("Freeport-Mcmoran","Mcmoran"))
print(is_valid_match("Jp Morgan Chase","Jpmorgan"))
print(difflib.SequenceMatcher(None,"Bank of america merril linch","BofA merril linch").ratio())
print(difflib.SequenceMatcher(None,"Curtiss-Wright","Curtis Wright").ratio())
print(difflib.SequenceMatcher(None,"Gordon Slater","Goldman Sachs").ratio())
print(difflib.SequenceMatcher(None,"Mes Inc","My Inc").ratio())

False
False
True
True
False
True
False
False
True
False
0.7111111111111111
0.8888888888888888
0.5384615384615384
0.7692307692307693


In [22]:
print(common_mixed_word("First BanCorp", "First BanCorp New"))
words = "RPM Inc".lower().replace("-"," ").split(" ")#for walmart and other
long_words = "RPM Technology Inc".lower().replace("-"," ").split(" ")#for walmart and other
from organisations_normalization import string_normalize        
print(long_name_match(words,long_words))
print(difflib.SequenceMatcher(None,"New Yorker","New York Times").ratio())

True
importing Jupyter notebook from organisations_normalization.ipynb
RPM Inc
44708
44708
44708
44708
55729
True
0.75


In [5]:
#getting sector data
#matching:
df_sectors = pd.read_csv("ls new handling normalized tickers holdings with industry.csv",index_col=False,header=0)

In [6]:

print(str(df_sectors.loc[df_sectors['Name'] == 'Bank Of America']['Industry'].iloc[0]))


Major Banks


In [25]:
#matching:
import json, itertools
from organisations_normalization import string_normalize
res_df = pd.DataFrame(columns=['Real company','Extracted company','Metrix','Comment','Article'])
df_keywords = pd.read_csv("dummy_keywords no copies.csv")

bigram_keywords = df_keywords['Sentence']

found_num = 0
not_found_num = 0
total = 0

labeled_companies = df_labels['Name']
news = df_news['News']
print(len(news))
print(len(labeled_companies))
headline = df_news['Keywords']
keywords = df_news['Headline']
#for i in range (50):
#    print(keywords[i])
#for i in range (50):
#    print(headline[i])
    #print(keywords[i].replace("{","").replace("}","").replace('"',"").split(","))

print(len(df_sectors))
COEFFS = {'contain_real': 0.9,'is_valid_match': 0.8,'is_both_unique': 0.8,'is_also_proper': 0.8,'is_abbreviation': 0.5,'is_also_person':0.8}
ARTICLES_NUM = len(labeled_companies)
extracted_companies = df_all_extracted_companies['Name']
perfect_companies = 0
contain_real_companies = 0

for idx in range (ARTICLES_NUM):
    labeled = labeled_companies[idx].split('\t')
    long_labeled = df_labels['Long name'][idx].split('\t')
    extracted = extracted_companies[idx].split('\t')
    keywords = bigram_keywords[idx]
    total += len(labeled)
    for labeled_elem, long_labeled_elem in zip(labeled, long_labeled):
        curr_sector = str(df_sectors.loc[df_sectors['Name'] == labeled_elem]['Industry'].iloc[0])
        curr_keywords = json.loads(bigram_keywords[idx].replace("'",'"')) # get list from ist str representation
        max_metrix = 0.0
        COMMENT = ""
        for extracted_elem in extracted:
            if(extracted_elem == "Not Avaliable" or extracted_elem.lower() == "nan"):
                continue
            labeled_str = labeled_elem.replace(" ","").replace("-","").lower()
            long_str = long_labeled_elem.replace(" ","").replace("-","").lower()
            extracted_str = extracted_elem.replace(" ","").replace("-","").lower()
            metrix = difflib.SequenceMatcher(None,labeled_str,extracted_str).ratio()
            long_metrix = difflib.SequenceMatcher(None,labeled_str,extracted_str).ratio()
            
            
            labeled_words = labeled_elem.lower().replace("-"," ").split(" ")#for walmart and other
            labeled_long_words = long_labeled_elem.lower().replace("-"," ").split(" ")#for walmart and other
            extracted_words = extracted_elem.lower().replace("-"," ").split(" ")

            if (long_metrix == 1.0 or metrix == 1.0 or max_metrix == 1.0 or labeled_str.replace("group","") == extracted_str or labeled_str.replace("&","and") == extracted_str or extracted_str.replace("&","and") == labeled_str):
                max_elem = extracted_elem
                max_metrix = 1.0
                COMMENT = ""
                perfect_companies += 1
                break
            if (is_extracted_contain_real(labeled_words,extracted_words) or (extracted_str.find(labeled_str.replace("group","")) !=-1 and len(extracted_words) > 1)):
                max_elem = extracted_elem
                max_metrix = 1.0
                contain_real_companies += 1
                COMMENT = "Extracted company contain real"
                break
            if(long_name_match(labeled_long_words,extracted_words)):
                max_elem = extracted_elem
                max_metrix = 1.0
                perfect_companies += 1
                break
            if(only_commonly_used_is_common(labeled_words,extracted_words) and metrix < 0.9):
                continue
            if (metrix > 0.3 and len(extracted_str) > 2 and (len(labeled_str) > 2) and (is_valid_match(labeled_elem,extracted_elem) or is_valid_match(extracted_elem,labeled_elem) or is_also_proper(labeled_elem,extracted_elem,nlp) or is_both_unique(labeled_words,extracted_words))):
                if(0.86 > max_metrix):
                    max_metrix = 0.86
                    max_elem = extracted_elem
            if (is_abbreviation(labeled_elem,extracted_elem) or is_abbreviation(extracted_elem,labeled_elem)):
                if(0.81 > max_metrix):
                    max_metrix = 0.81
                    max_elem = extracted_elem
            if (is_complicated_abbreviation(labeled_elem,extracted_elem) or is_complicated_abbreviation(extracted_elem,labeled_elem)):
                if(0.83 > max_metrix):
                    max_metrix = 0.83
                    max_elem = extracted_elem
            if common_mixed_word(labeled_words,extracted_words):
                if(0.85 > max_metrix):
                    max_metrix = 0.85
                    max_elem = extracted_elem
            if metrix > max_metrix:
                max_metrix = metrix
                max_elem = extracted_elem
                COMMENT = ""
                isOkay = True
                for elem in curr_keywords:
                    if(not isOkay):
                        break
                    elem = string_normalize(elem)
                    if(labeled_elem.find(elem)!= -1):
                        max_metrix = min(1.0, max_metrix + 0.15)
                        COMMENT = "Common keywords"
                        isOKay = False
                for word in labeled_words:
                    if word.lower() in commonly_used_words:
                        continue
                    if word in extracted_words  and curr_sector.find(word)!=-1 and len(word) > 2:
                        max_metrix = min(1, max_metrix + 0.10)
                        COMMENT = "Common keywords"
                        print(extracted_elem," + ", labeled_elem, " + ", curr_sector, " + ",elem)
                        break
        if (max_metrix > 0.95)or(max_metrix > 0.8  and len(labeled_elem) > 2 and len(max_elem) > 2):      
            res_df.loc[found_num]=[labeled_elem,max_elem,max_metrix,COMMENT,news[idx]]
            found_num += 1
        else:
            not_found_num += 1
    if(idx % 2000 == 0):
        print(idx)
            #print(labeled_elem,"+", extracted," best: ",max_elem," :idx = ",idx)
print("not found", not_found_num)
print("found", found_num)
print("total",total)
print("found_res = ",found_num/total)
print("perfect_res = ",perfect_companies/total)
print("contained_res = ",contain_real_companies/total)
print(res_df.count())
res_df.drop_duplicates(subset=['Real company','Extracted company'],inplace=True)
print(res_df.count())
res_df.to_csv("with keyword results/long_str_new_handling.csv",index=False)


44708
44708
6161
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
not found 2907
found 61812
total 64719
found_res =  0.9550827423167849
perfect_res =  0.8066101144949706
contained_res =  0.06997944962066781
Real company         61812
Extracted company    61812
Metrix               61812
Comment              61812
Article              61812
dtype: int64
Real company         2724
Extracted company    2724
Metrix               2724
Comment              2724
Article              2724
dtype: int64


In [27]:
#only nasdaq:
#93,7% - bert ->94,18%
#90.9% - spacy
#52.3% - nltk
#95% - flair

In [28]:
#add  to normalization:
#if(elem.find(" ")!=-1 and extracted_elem.find(" ")!=-1):# delete words except first with len < 3 if both companies have more than 1 word
#                labeled_elem = re.sub(r' \w{1,2}\b', '', labeled_elem).strip()
#                extracted_elem = re.sub(r' \w{1,2}\b', '', extracted_elem).strip()

In [29]:
#94.7% (73,6% perfect)- spacy
#95,7% (73,4% perfect) - bert

#no the
#found_res =  0.9525623053925643
#perfect_res =  0.7489052362580789
#contained_res =  0.1019215739787374


#strict
#found_res =  0.945615362667626
#perfect_res =  0.7489052362580789
#contained_res =  0.1019215739787374

#extended
#found_res =  0.9345622805820546
#perfect_res =  0.7300492488618179
#contained_res =  0.10829787497984146

#extended large
#found_res =  0.956159829303693
#perfect_res =  0.7513862872312712
#contained_res =  0.10868243788068625

#found_res =  0.9563583133815484
#perfect_res =  0.7513862872312712
#contained_res =  0.10868243788068625

#large + alice strict
#found_res =  0.964357383223735
#perfect_res =  0.8078941681331234
#contained_res =  0.05939921583155815

#news handled (not remove &)
#found_res =  0.9623603578547258
#perfect_res =  0.8097003971013149
#contained_res =  0.0542035569152799

#used keywords
#found_res =  0.9624994205720113
#perfect_res =  0.8117245322084705
#contained_res =  0.05409539702405785

#new normalization :
#found_res =  0.9640445618751835
#perfect_res =  0.824471947959641
#contained_res =  0.05247299865572706

#found_res =  0.9595636520959842
#perfect_res =  0.8135323475331819
#contained_res =  0.07673171711553022